# <font color=darkblue> Machine Learning model deployment with Flask framework on Heroku</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with flask framework on heroku.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import scipy.stats as zscore
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier

### 2. Load the dataset

In [5]:
df = pd.read_csv(r"C:\Users\preet\Downloads\car+data.csv", encoding='unicode_escape')
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [6]:
df.shape

(301, 9)

In [7]:
df.info

<bound method DataFrame.info of     Car_Name  Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type  \
0       ritz  2014           3.35           5.59       27000    Petrol   
1        sx4  2013           4.75           9.54       43000    Diesel   
2       ciaz  2017           7.25           9.85        6900    Petrol   
3    wagon r  2011           2.85           4.15        5200    Petrol   
4      swift  2014           4.60           6.87       42450    Diesel   
..       ...   ...            ...            ...         ...       ...   
296     city  2016           9.50          11.60       33988    Diesel   
297     brio  2015           4.00           5.90       60000    Petrol   
298     city  2009           3.35          11.00       87934    Petrol   
299     city  2017          11.50          12.50        9000    Diesel   
300     brio  2016           5.30           5.90        5464    Petrol   

    Seller_Type Transmission  Owner  
0        Dealer       Manual      0  
1  

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [8]:
print("Duplicate Records : ",len(df[df.duplicated()]))

Duplicate Records :  2


In [9]:
df.drop_duplicates(inplace=True)
df.shape

(299, 9)

### 5. Drop the columns which you think redundant for the analysis.

In [10]:
df = df.drop(columns=(["Owner","Present_Price"]), axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 300
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       299 non-null    object 
 1   Year           299 non-null    int64  
 2   Selling_Price  299 non-null    float64
 3   Kms_Driven     299 non-null    int64  
 4   Fuel_Type      299 non-null    object 
 5   Seller_Type    299 non-null    object 
 6   Transmission   299 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 18.7+ KB


In [11]:
miss_val_per = df.isnull().sum()*100/len(df)
print(miss_val_per)

Car_Name         0.0
Year             0.0
Selling_Price    0.0
Kms_Driven       0.0
Fuel_Type        0.0
Seller_Type      0.0
Transmission     0.0
dtype: float64


### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [12]:
from datetime import date

df['age_of_the_car'] = date.today().year - df['Year']
df[['Year','age_of_the_car']].head()

,Year,age_of_the_car
0,2014,8
1,2013,9
2,2017,5
3,2011,11
4,2014,8


In [13]:
df = df.drop("Year", axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 0 to 300
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Car_Name        299 non-null    object 
 1   Selling_Price   299 non-null    float64
 2   Kms_Driven      299 non-null    int64  
 3   Fuel_Type       299 non-null    object 
 4   Seller_Type     299 non-null    object 
 5   Transmission    299 non-null    object 
 6   age_of_the_car  299 non-null    int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 18.7+ KB


### 7. Encode the categorical columns

In [16]:
category = ['Fuel_Type','Seller_Type','Transmission']
lbl_encode=LabelEncoder()

for i in category:
    df[i]=df[[i]].apply(lbl_encode.fit_transform)
    
df.head()

,Car_Name,Selling_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,age_of_the_car
0,ritz,3.35,27000,2,0,1,8
1,sx4,4.75,43000,1,0,1,9
2,ciaz,7.25,6900,2,0,1,5
3,wagon r,2.85,5200,2,0,1,11
4,swift,4.60,42450,1,0,1,8


In [17]:
features = pd.get_dummies(df)

In [18]:
features.iloc[:,:].head(5)

,Selling_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,age_of_the_car,Car_Name_800,Car_Name_Activa 3g,Car_Name_Activa 4g,Car_Name_Bajaj ct 100,...,Car_Name_land cruiser,Car_Name_omni,Car_Name_ritz,Car_Name_s cross,Car_Name_swift,Car_Name_sx4,Car_Name_verna,Car_Name_vitara brezza,Car_Name_wagon r,Car_Name_xcent
0,3.35,27000,2,0,1,8,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,4.75,43000,1,0,1,9,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,7.25,6900,2,0,1,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.85,5200,2,0,1,11,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4.60,42450,1,0,1,8,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [19]:
labels = np.array(features['Selling_Price'])
feature_list = list(features.columns)
features = np.array(features)

### 8. Separate the target and independent features.

In [20]:
df.corrwith(df['Selling_Price'])

Selling_Price     1.000000
Kms_Driven        0.028566
Fuel_Type        -0.500292
Seller_Type      -0.553851
Transmission     -0.348869
age_of_the_car   -0.234369
dtype: float64

### 9. Split the data into train and test.

In [27]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
print("Training Feature Shape : ",train_features.shape)
print("\nTraining Labels Shape : ",train_labels.shape)
print("\nTesting Features Shape : ",test_features.shape)
print("\nTesting Labels Shape : ",test_labels.shape)

Training Feature Shape :  (224, 104)

Training Labels Shape :  (224,)

Testing Features Shape :  (75, 104)

Testing Labels Shape :  (75,)


In [28]:
baseline_preds = test_features[:, feature_list.index('Selling_Price')]
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors),2))

Average baseline error:  0.0


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [29]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_features, train_labels)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [30]:
predictions = rf.predict(test_features)
errors = abs(predictions - test_labels)
print('Mean Absolute Error:', round(np.mean(errors),2), 'degress.')

Mean Absolute Error: 0.17 degress.


In [31]:
mape = 100 * (errors/test_labels)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy,2), '%.')

Accuracy: 98.15 %.


### 11. Create a pickle file with an extension as .pkl

In [32]:
import pickle
pickle.dump(rf, open('model1.pkl','wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html >
<head>
  <meta charset="UTF-8">
  <title>ML Deployment</title>
  <link href='https://fonts.googleapis.com/css?family=Pacifico' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Arimo' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Hind:300' rel='stylesheet' type='text/css'>
<link href='https://fonts.googleapis.com/css?family=Open+Sans+Condensed:300' rel='stylesheet' type='text/css'>
<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  
</head>
<body>
 <div class="login">
	<h1>Iris Flower Species Classifier</h1>

     <!-- Main Input For Receiving Query to our ML -->
    <form action="{{ url_for('predict')}}"method="post">
    	 
    	<input type="text" name="sepal_length" placeholder="Sepal Length (cm)" required="required" />
        <input type="text" name="sepal_width" placeholder="Sepal Width (cm)" required="required" />
		<input type="text" name="petal_length" placeholder="Petal Length (cm)" required="required" />
		<input type="text" name="petal_width" placeholder="Petal Width (cm)" required="required" />

        <button type="submit" class="btn btn-primary btn-block btn-large">Predict</button>
    </form>

   <br>
   <br>
   {{ prediction_text }}
 </div>
</body>
</html>

### b) Create app.py file and write the predict function

In [ ]:
# importing necessary libraries and functions
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__) #Initialize the flask App
model = pickle.load(open('model.pkl', 'rb')) # loading the trained model

@app.route('/') # Homepage
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    
    # retrieving values from form
    init_features = [float(x) for x in request.form.values()]

    final_features = [np.array(init_features)]
    prediction = model.predict(final_features) # making prediction


    return render_template('index.html', prediction_text='Predicted Class: {}'.format(prediction)) # rendering the predicted result

if __name__ == "__main__":
    app.run(debug=True)

### 13. Deploy your app on Heroku. (write commands for deployment)

In [ ]:
heroku git: remote -a preethamup
git push heroku master

### 14. Paste the URL of the heroku application below, and while submitting the solution submit this notebook along with the source code.

In [ ]:
https://preethamup.herokuapp.com/

### Happy Learning :)